# Prequisites

In [1]:
# Set Project Directory
import os
from pathlib import Path

colab = True  # Set to True if using Google Colab
if colab:
    !git clone https://github.com/soroushdty/pdm.git
    os.chdir("pdm")

PROJECT_DIR = Path.cwd()
print("Project Directory:", PROJECT_DIR)

Cloning into 'pdm'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 908 (delta 48), reused 40 (delta 23), pack-reused 826 (from 1)
Receiving objects: 100% (908/908), 1.74 MiB | 6.44 MiB/s, done.
Resolving deltas: 100% (548/548), done.
Project Directory: /content/pdm


In [2]:
# Set up logging
import logging
logging.basicConfig(
    filename = PROJECT_DIR / "log.txt",
    level = logging.DEBUG,
    datefmt = '%H:%M:%S  %m-%d-%y',
    format = '%(asctime)s  %(levelname)s:  %(message)s',
    force = True,
    filemode='w')
logging.info(f"START LOGGING AT: {PROJECT_DIR}")

In [3]:
# Load configuration
from funcs.config_loader import load_config
import json
CONFIG_PATH = PROJECT_DIR / "config.json"
try:
  cfg = load_config(CONFIG_PATH)
  logging.info("Configuration loaded successfully.")
except Exception as e:
  logging.error(f"ERROR: {e}")

In [4]:
# Install Requirements
REQUIREMENTS_PATH = PROJECT_DIR / "requirements.txt"
from funcs.requirements_utils import requirements_utils
installed = requirements_utils(REQUIREMENTS_PATH)
print("Installed Libraries:", installed)

Installed Libraries: ['numpy', 'pandas', 'matplotlib', 'seaborn', 'scipy', 'torch', 'openpyxl', 'joblib', 'scikit-learn', 'imbalanced-learn', 'iterative-stratification']


In [5]:
# import shutil
import numpy as np
import pandas as pd
import torch
from funcs.preprocessing import run as preprocessing
from funcs.mean_pooling import mean_pooling
from funcs.sanitize_col_name import sanitize_col_name
from funcs. compute_embeddings import compute_embeddings
from funcs.train_ensemble_pipeline import train_ensemble_pipeline
from funcs.get_dataset_from_npz import get_dataset_from_npz
logging.debug("All imports done.")

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.debug(f"Using device: {device}")

# Preprocessing

In [7]:
p = preprocessing(cfg)
logging.debug("Preprocessing done.")
for k,v in p.items():
  for k1,v1 in v.items():
    print(f"{k}: {k1}: {v1}")

merged_counts: train: 879
merged_counts: test: 120
paths: output_dir: /content/pdm/output
paths: train_map: /content/pdm/output/merge_map_train.json
paths: test_map: /content/pdm/output/merge_map_test.json
paths: train_csv: /content/pdm/output/train.csv
paths: test_csv: /content/pdm/output/test.csv
shapes: train_raw: (1758, 14)
shapes: test_raw: (240, 14)
shapes: train_final: (868, 13)
shapes: test_final: (89, 13)


In [8]:
train_csv = pd.read_csv(PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'train.csv')
test_csv = pd.read_csv(PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'test.csv')
train_map = PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'merge_map_train.json'
test_map = PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'merge_map_test.json'
print('Shape of train.csv', train_csv.shape)
print('Shape of test.csv', test_csv.shape)
print('Path of train_map:', train_map)
print('Path of test_map:', test_map)

Shape of train.csv (868, 13)
Shape of test.csv (89, 13)
Path of train_map: /content/pdm/output/merge_map_train.json
Path of test_map: /content/pdm/output/merge_map_test.json


# y npz files generation

In [9]:
col_included = [c for c in cfg['classes'] if c not in cfg['other_class']]
z_train = train_csv.loc[:, col_included]
yy_train = z_train.to_numpy()
z_test = test_csv.loc[:, col_included]
yy_test = z_test.to_numpy()
y_file_name = PROJECT_DIR / cfg['DIR_OUTPUT'] / 'y.npz'
logging.debug(f"y npz file path: {y_file_name}")
np.savez_compressed(y_file_name, **{'train': yy_train, 'test': yy_test})

In [10]:
y_train = np.load(y_file_name)['train']
y_test = np.load(y_file_name)['test']
print("Shape of y_test:", y_train.shape)
print("Shape of y_train:",y_test.shape)

Shape of y_test: (868, 4)
Shape of y_train: (89, 4)


# embedding

In [11]:
llm_example = cfg['llms'][0]
BATCH_SIZE = cfg['BATCH_SIZE']
print(llm_example)
print(BATCH_SIZE)

sentence-transformers/all-MiniLM-L6-v2
128


In [12]:
items_train = train_csv['Item']
items_test = test_csv['Item']
items = pd.concat([items_train, items_test]).drop_duplicates().sort_values()
embedding = compute_embeddings(llm_example, items.to_list(), BATCH_SIZE)
print('\n', 'Shape of embedding: ', embedding.shape)

Computing embeddings for sentence-transformers/all-MiniLM-L6-v2...
Attempting to load sentence-transformers/all-MiniLM-L6-v2 as SentenceTransformer...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Success with SentenceTransformer.

 Shape of embedding:  (651, 384)


In [13]:
sanitized = sanitize_col_name(llm_example)
print('LLM: ', sanitized)
embedding_file_name = PROJECT_DIR / cfg['DIR_OUTPUT'] / f'embdedding-{sanitized}-item.npz'
item_embedding_dict = {item: embedding[i] for i, item in enumerate(items.to_list())}
np.savez_compressed(embedding_file_name, **item_embedding_dict)
print('Saved to: ', embedding_file_name)

LLM:  sentence_transformers__all_MiniLM_L6_v2
Saved to:  /content/pdm/output/embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz


# base model

In [14]:
x_path = PROJECT_DIR / cfg["DIR_OUTPUT"] /'embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz'
x = np.load(x_path)
print('Path of x.npz: ', x_path)
print("len(x.keys): ", len(x.keys()))

Path of x.npz:  /content/pdm/output/embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz
len(x.keys):  651


In [15]:
y_file_name = PROJECT_DIR / cfg['DIR_OUTPUT'] / 'y.npz'
y_train = np.load(y_file_name)['train']
y_test = np.load(y_file_name)['test']
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

X_train, X_test = get_dataset_from_npz(x, y_train, y_test, train_map, test_map)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of y_train: (868, 4)
Shape of y_test: (89, 4)
Shape of X_train: (868, 384)
Shape of X_test: (89, 384)


In [16]:
import logging, traceback
def main():
  try:
    print("Global X_train overall variance:", np.var(X_train))
    print("First 10 feature variances:", np.var(X_train, axis=0)[:10])
    train_ensemble_pipeline(X_train, y_train, X_test, y_test, cfg)
    print("Pipeline executed successfully.")
  except Exception as e:
    logging.info(f"CRITICAL ERROR: {e}")
    traceback.print_exc()


if __name__ == "__main__":
  main()

Global X_train overall variance: 0.0
First 10 feature variances: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (462, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (462, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


...

[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (694, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (694, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:789: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Pipeline executed successfully.


In [ ]:



# Output is:

# dir
# ./<cfg["ensemble_sub_dir"]>/
# ./<cfg["ensemble_sub_dir"]>/train_cv_report/
# ./<cfg["ensemble_sub_dir"]>/test_report/

# file
# ./<cfg["ensemble_sub_dir"]>/ensemble_model.pkl
# ./<cfg["ensemble_sub_dir"]>/train_cv_report/Train_CV_Metrics.csv
# ./<cfg["ensemble_sub_dir"]>/Ensemble_Metrics.csv
# ./<cfg["ensemble_sub_dir"]>/train_cv_report/ (plots for CV/OOF)
# ./<cfg["ensemble_sub_dir"]>/test_report/ (plots for test predictions)

Global X_train overall variance: 0.0
First 10 feature variances: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[09:18:21 02-02-26] INFO: Starting Training on X_train: (868, 384)...
[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (462, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (462, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


...

[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (694, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (694, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:789: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


[09:19:12 02-02-26] INFO: Saved metrics to ./ensemble/train_cv_report/Train_CV_Metrics.csv
[09:19:13 02-02-26] INFO: Saved metrics to ./ensemble/Ensemble_Metrics.csv
[09:19:14 02-02-26] INFO: Pipeline Complete.
Pipeline executed successfully.


# NEXT

## Embed Context

## Fusion

## Prediction

In [ ]:
# X_old = 'Alpha-1-fetoprotein (AFP) and choriogonadotropin (HCG) tumor markers'
# X_old_embedding = compute_embeddings(llm_example, [X_old], BATCH_SIZE)
# print(X_old_embedding.shape)

# context = pd.read_csv(PROJECT_DIR / cfg['DIR_INPUT'] / 'patient_summaries.csv')
# context_example = context.iloc[10]
# context_embedding = compute_embeddings(llm_example, context_example.to_list(), BATCH_SIZE)
# print(context_embedding.shape)


# averaged_context_embedding = np.mean(context_embedding, axis=0)
# single_X_old_embedding = X_old_embedding.squeeze(0)
# fusion_intermed =

# print('Shape of fusion_intermed: ', fusion_intermed.shape)

Computing embeddings for sentence-transformers/all-MiniLM-L6-v2...
Attempting to load sentence-transformers/all-MiniLM-L6-v2 as SentenceTransformer...
[09:51:43 02-02-26] INFO: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Success with SentenceTransformer.
(1, 384)
Computing embeddings for sentence-transformers/all-MiniLM-L6-v2...
Attempting to load sentence-transformers/all-MiniLM-L6-v2 as SentenceTransformer...
[09:51:44 02-02-26] INFO: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Success with SentenceTransformer.
(9, 384)
Shape of fusion_intermed:  (768,)


In [ ]:
# import joblib
# import torch
# from cls.EnsemblePredictor import EnsemblePredictor
# from cls.Preprocessor import Preprocessor

# # Temporarily override torch.load to always map to CPU
# _original_torch_load = torch.load

# def _torch_load_cpu(*args, **kwargs):
#     kwargs['map_location'] = torch.device('cpu')
#     return _original_torch_load(*args, **kwargs)

# torch.load = _torch_load_cpu

# try:
#     ens_model = joblib.load('/content/ensemble_model.pkl')
#     # Explicitly set the device of the loaded model to CPU
#     ens_model.device = torch.device('cpu')
# finally:
#     # Restore original torch.load after loading
#     torch.load = _original_torch_load

# # Prepare input for probs_old: it needs to be 768 features (item + context) and 2D.
# # We will use the same averaged_context_embedding for X_old.
# X_old_input = np.concatenate([averaged_context_embedding, single_X_old_embedding]).reshape(1, -1)
# probs_old = ens_model.predict_proba(X_old_input)

# # Prepare input for probs_new: fusion_intermed is already 768 features, but needs to be 2D.
# probs_new = ens_model.predict_proba(fusion_intermed.reshape(1, -1))

In [ ]:
# print(probs_old)
# print(probs_new)

[[0.09580949 0.2737596  0.30702    0.33254802]]
[0.8, 0.56332, 0.843, 0.1323]
